In [140]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.corpus import *
from nltk.tokenize import sent_tokenize 
from nltk.tokenize import word_tokenize
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
n_instances = 500
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]



In [54]:
corpus_root=r'C:\Users\kimyujin\Desktop\2020 3-2학기\인공지능\sentence_tokenized'
gniCorpus=PlaintextCorpusReader(corpus_root,'.*\.txt',encoding='utf-8')



In [187]:
def posORnegORneufunc():
    train_subj_docs = subj_docs[:450]
    test_subj_docs = subj_docs[450:500]
    train_obj_docs = obj_docs[:450]
    test_obj_docs = obj_docs[450:500]
    training_docs = train_subj_docs+train_obj_docs
    testing_docs = test_subj_docs+test_obj_docs
    sentim_analyzer = SentimentAnalyzer()
    all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])

    unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)

    sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)

    training_set = sentim_analyzer.apply_features(training_docs)
    test_set = sentim_analyzer.apply_features(testing_docs)
    
    trainer = NaiveBayesClassifier.train
    classifier = sentim_analyzer.train(trainer, training_set)

    for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
        print('{0}: {1}'.format(key, value))
    
    sid = SentimentIntensityAnalyzer()
    
    
    sentRaw=[]
    allList=gniCorpus.fileids()
    scoreDic={}

    for k in range(0,len(gniCorpus.fileids())):
        sentRaw.append(sent_tokenize(gniCorpus.raw(allList[k])))


    print('\n분류 중 ... 잠시 기다려 주세요.\n')
    for i in range(0,len(sentRaw)):
        totalCompound=0

        #print((sentRaw[i][0].split('Title: ')[1]).split('\n\n')[0])
        text = word_tokenize((sentRaw[i][0].split('Title: ')[1]).split('\n\n')[0])
        title_pos=nltk.pos_tag(text)
        NNP_NNPS=[x[0] for x in title_pos if x[1]=='NNP' or x[1]=='NNPS']
        #print(NNP_NNPS)
        for j in range(0,len(sentRaw[i])):
            ss = sid.polarity_scores(sentRaw[i][j])
            totalCompound=totalCompound+ss['compound']

        if(totalCompound>0):
            for i in NNP_NNPS:
                if (i not in scoreDic):
                    scoreDic[i]=[1,0]
                else:
                    scoreDic[i][0]+=1

        elif(totalCompound<0):
            for i in NNP_NNPS:
                if (i not in scoreDic):
                    scoreDic[i]=[0,1]
                else:
                    scoreDic[i][1]+=1
                    
                    
    neg_list=[]
    pos_list=[]
    result_list=[]
    for i in scoreDic.values():
        if(i[0]>i[1]):
            result_list.append('more Positive')
        elif(i[0]<i[1]):
            result_list.append('more Negative')
        else:
            result_list.append('almost Neural')
        pos_list.append(i[0])
        neg_list.append(i[1])

    df = pd.DataFrame()
    df['word'] = scoreDic.keys()
    df['pos'] = pos_list
    df['neg'] = neg_list
    df['result'] = result_list
    df.index = df.index + 1
    pd.set_option('display.max_rows', df.shape[0]+1)
    
    display(df)





    




In [188]:
def letstart():
    posORnegORneufunc()

Training classifier
Evaluating NaiveBayesClassifier results...
Accuracy: 0.94
F-measure [obj]: 0.9411764705882353
F-measure [subj]: 0.9387755102040818
Precision [obj]: 0.9230769230769231
Precision [subj]: 0.9583333333333334
Recall [obj]: 0.96
Recall [subj]: 0.92
분류 중 ... 잠시 기다려 주세요.



,word,pos,neg,result
1,Survey,1,0,more Positive
2,Applications,1,0,more Positive
3,NGS,1,0,more Positive
4,Whole-Genome,1,1,almost Neural
5,Sequencing,12,7,more Positive
6,Expression,8,4,more Positive
7,Profiling,1,1,almost Neural
8,PKD2-Associated,1,0,more Positive
9,Genes,9,2,more Positive
10,Autosomal,1,0,more Positive
